## 한글 폰트 설치

In [1]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab 경우 Runtime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

## matplotlib 시각화

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용


## load dataset

In [3]:
import pandas as pd
df_FSSH = pd.read_csv('fruitstore_saleshistory.csv')
df_FSSH[:3]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05


## 범주형 / 연속형 : 범주형 따른 관계 확인

## 연속형 데이터 정규성 검정

In [21]:
# stats.anderson을 사용한 정규 분포 검정
result = stats.anderson(df_FSSH['구매금액'].dropna())
print('Test Statistic: %.3f' % result.statistic)
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < cv:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따름 (검정 통계량 %.3f < 임계값 %.3f)' % (sl, result.statistic, cv))
    else:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 %.3f >= 임계값 %.3f)' % (sl, result.statistic, cv))

Test Statistic: 80887.237
15.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.576)
10.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.656)
5.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.787)
2.5% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.918)
1.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 1.092)


### 연속형이 정규 분포이며, 등분산 일때

### 예제 가설 : 과일 종류, 평균 판매 금액
- 분석 내용 : 각 과일 종류(범주형)별 평균 판매 금액(연속형)에 차이가 있는지 분석합니다.
  + 귀무 가설: 평균 차이가 없다.
  + 대립 가설: 평균 차이가 있다.

### 등분산 검증

In [4]:
# 필요 항목 확인
fruit_df = df_FSSH[df_FSSH['물품중분류'].str.contains('과일')]
fruit_df[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
6,2018-01-02,화,1,1,743192512,정상회원,채소,과일채소,딸기,딸기/(500g),1.0,수지구,성복동,여,51.0,50대,17038,매장C,NaN,10:12


In [5]:
# 과일 종류 확인
fruit_list = fruit_df['물품소분류'].unique().tolist()
fruit_list

['사과',
 '딸기',
 '건대추',
 '귤(조생)',
 '배',
 '산딸기',
 '완숙토마토',
 '단감',
 '참다래',
 '레몬',
 '블루베리',
 '곶감',
 '귤만감',
 '방울토마토',
 '냉동건조과일',
 '홍시용감(대봉)',
 '냉동딸기',
 '냉동메론',
 '귤(비가림)',
 '참외',
 '수박',
 '청매실',
 '황매실',
 '복숭아',
 '자두']

### Kruskal Test
- 연속형이 비정규 분포 이면서 범주형에 따른 항목이 등분산, 항목 갯수가 3개 이상

In [20]:
sale_by_fruit_list = list()

for fruit in fruit_list:
  condition = f"물품소분류 == '{fruit}'"
  fruit_data = fruit_df.query(condition).copy()
  fruit_data = fruit_data.dropna(subset=['구매금액'])
  sale_amount = fruit_data['구매금액']
  sale_by_fruit_list.append(sale_amount)
sale_by_fruit_list[0][:3]

0      22207
232    18953
689    18953
Name: 구매금액, dtype: int64

In [22]:
import scipy.stats as stats
stats.kruskal(*sale_by_fruit_list)

KruskalResult(statistic=11484.596126606793, pvalue=0.0)

### 분석결과
- 통계 결론 : 연속형 데이터의 정규성을 Anderson Test로 검사 했을 때 비졍규 분포를 띔. 범주형 데이터가 3개 이상이므로 Kruskal Test 시행. p-value 0.05 이하이므로 대립 가설 채택.
- 사실 결론 : 각 과일 종류(범주형)별 평균 판매 금액(연속형)에 차이가 있다.
